In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv('E:\Codes\Projects\IPl predictor\ipl_data.csv')

Setting the required rate to 0 when match is over

In [3]:
data[data['balls_left'] < 0]

,batting_team,bowling_team,city,runs_left,balls_left,wickets_left,target_runs,RR,RRR,result
494,Delhi Capitals,Chennai Super Kings,Chennai,28.0,-2,2,168.0,6.885246,-84.0,0
558,Delhi Capitals,Sunrisers Hyderabad,Delhi,10.0,-2,4,198.0,9.245902,-30.0,0
787,Mumbai Indians,Punjab Kings,Mumbai,2.0,-1,0,190.0,9.322314,-12.0,0
1356,Royal Challengers Bengaluru,Mumbai Indians,Port Elizabeth,17.0,-1,3,158.0,6.991736,-102.0,0
1532,Rajasthan Royals,Delhi Capitals,Dubai,14.0,-1,2,162.0,7.338843,-84.0,0
...,...,...,...,...,...,...,...,...,...,...
113033,Kolkata Knight Riders,Delhi Capitals,Delhi,56.0,-1,1,220.0,8.132231,-336.0,0
113394,Lucknow Super Giants,Chennai Super Kings,Chennai,19.0,-1,3,218.0,9.867769,-114.0,0
113510,Punjab Kings,Mumbai Indians,Mumbai,5.0,-1,1,175.0,8.429752,-30.0,0
113720,Punjab Kings,Rajasthan Royals,Chandigarh,44.0,-2,2,178.0,6.590164,-132.0,0


In [4]:
data['RRR'] = data['RRR'].fillna(0)

Not calculating when balls left == 0

In [5]:
data = data[data['balls_left']>0]

Setting negative runs_left to 0

In [6]:
data.loc[data['runs_left'] < 0, 'runs_left'] = 0


In [7]:
data['RRR'] = (data['runs_left'] *6 )/ data['balls_left']
data['RR'] = ((data['target_runs'] - data['runs_left'])*6) / (120 - data['balls_left'])

In [8]:
data.describe()

,runs_left,balls_left,wickets_left,target_runs,RR,RRR,result
count,114101.000000,114101.000000,114101.000000,114101.000000,114101.000000,114101.000000,114101.000000
mean,95.027300,62.813069,7.517410,169.705927,7.603287,10.888222,0.372354
std,50.985272,33.243610,2.151392,30.615240,2.345944,13.236628,0.483434
min,0.000000,1.000000,0.000000,68.000000,0.000000,0.000000,0.000000
25%,55.000000,35.000000,6.000000,150.000000,6.400000,7.322034,0.000000
50%,94.000000,63.000000,8.000000,169.000000,7.631068,9.076923,0.000000
75%,133.000000,92.000000,9.000000,189.000000,8.880000,11.294118,1.000000
max,287.000000,119.000000,10.000000,288.000000,36.000000,714.000000,1.000000


In [10]:
data = data.drop((['batting_team', 'bowling_team']), axis=1)

In [11]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [12]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 11)

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
trnsfrm = ColumnTransformer([
    ('trnsfrm', OneHotEncoder(drop = 'first', sparse_output=False), ['city']), 
    ('scaler', StandardScaler(), ['runs_left', 'balls_left', 'target_runs']),
], remainder = 'passthrough')

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
pipe = Pipeline([
    ('trnsfrm', trnsfrm),
    ('model', LogisticRegression(solver = 'liblinear', class_weight='balanced'))
])


In [17]:
pipe.fit(x_train, y_train)

C:\Users\tyagi\AppData\Roaming\Python\Python310\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('trnsfrm',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('trnsfrm',
                                                  OneHotEncoder(drop='first',
                                                                sparse_output=False),
                                                  ['city']),
                                                 ('scaler', StandardScaler(),
                                                  ['runs_left', 'balls_left',
                                                   'target_runs'])])),
                ('model',
                 LogisticRegression(class_weight='balanced',
                                    solver='liblinear'))])

In [18]:
y_pred = pipe.predict(x_test)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7397572411375487

In [20]:
import pickle
pickle.dump(pipe, open('E:\Codes\Projects\IPl predictor\pipe2.pkl', 'wb'))